# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/city_weather_data.csv')
df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pangnirtung,66.15,-65.71,21.20,79.0,75.0,6.93,CA,1.603725e+09
1,Atuona,-9.80,-139.03,78.51,74.0,16.0,22.50,PF,1.603725e+09
2,Alofi,-19.06,-169.92,73.89,87.0,100.0,22.50,NU,1.603725e+09
3,Vaini,-21.20,-175.20,69.80,94.0,75.0,8.05,TO,1.603725e+09
4,Busselton,-33.65,115.33,57.99,71.0,0.0,3.49,AU,1.603725e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [7]:
fig = gmaps.figure()
locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
hotel_df = df.loc[(df['Max Temp'] > 70) & (df['Max Temp'] < 80) & (df['Cloudiness'] == 0) & (df['Wind Speed'] < 10)].copy()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Tangi,34.30,71.65,71.60,35.0,0.0,9.17,PK,1.603725e+09
178,Teknāf,20.86,92.31,79.38,83.0,0.0,4.12,BD,1.603725e+09
232,Khanpur,28.65,70.66,76.28,18.0,0.0,5.82,PK,1.603726e+09
270,Mihijam,23.85,86.88,73.09,47.0,0.0,4.81,IN,1.603726e+09
284,Viedma,-40.81,-63.00,73.99,25.0,0.0,8.01,AR,1.603726e+09
313,Marrakesh,31.63,-8.01,77.00,33.0,0.0,4.70,MA,1.603726e+09
322,Sur,22.57,59.53,76.08,64.0,0.0,7.36,OM,1.603726e+09
389,Mananjary,-21.22,48.33,73.54,74.0,0.0,7.40,MG,1.603726e+09
395,Srīvardhan,18.03,73.02,79.27,84.0,0.0,1.90,IN,1.603726e+09
420,Marzuq,14.40,46.47,74.77,18.0,0.0,6.20,YE,1.603726e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df['Hotel Name'] = ''
locations = hotel_df[['Lat', 'Lng']]
hotel_df = hotel_df.reset_index(drop=True)


In [37]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'rankby': 'distance',
    'type': 'lodging',
    'key': g_key,
    'keyword': 'hotel'
}

for index, row in hotel_df.iterrows():
    location = f'{row["Lat"]},{row["Lng"]}'
    params['location'] = location
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(SyntaxError, IndexError):
        print('whoops')

whoops


In [38]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Tangi,34.30,71.65,71.60,35.0,0.0,9.17,PK,1.603725e+09,Kanewar Fish Center
1,Teknāf,20.86,92.31,79.38,83.0,0.0,4.12,BD,1.603725e+09,Alo Resort
2,Khanpur,28.65,70.66,76.28,18.0,0.0,5.82,PK,1.603726e+09,VIP shama biryani khanpur
3,Mihijam,23.85,86.88,73.09,47.0,0.0,4.81,IN,1.603726e+09,OYO 28258 Hotel Rajeswari
4,Viedma,-40.81,-63.00,73.99,25.0,0.0,8.01,AR,1.603726e+09,Alta Patagonia Apart
5,Marrakesh,31.63,-8.01,77.00,33.0,0.0,4.70,MA,1.603726e+09,2Ciels Luxury Boutique Hôtel & Spa
6,Sur,22.57,59.53,76.08,64.0,0.0,7.36,OM,1.603726e+09,Arabian Sea Motel
7,Mananjary,-21.22,48.33,73.54,74.0,0.0,7.40,MG,1.603726e+09,Sorafa Hotel
8,Srīvardhan,18.03,73.02,79.27,84.0,0.0,1.90,IN,1.603726e+09,Lambe Holiday Inn Resort
9,Marzuq,14.40,46.47,74.77,18.0,0.0,6.20,YE,1.603726e+09,


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))